<a href="https://colab.research.google.com/github/khatana706/DATA-SCIENCE/blob/train/Copy_of_05_01_2021_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive._mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
%cd /gdrive/My Drive/darknet
!pip install -q yolov4==2.1.0

/gdrive/My Drive/darknet


In [ ]:
import cv2
import os
import numpy as np
import tensorflow as tf
import time
from yolov4.tf import YOLOv4
from PIL import Image
from PIL.Image import fromarray as fa
from IPython.display import clear_output
from random import shuffle
import matplotlib.pyplot as plt
from time import sleep
from os import listdir
from os.path import join
import shutil

In [ ]:
folder = "accessories_all_together_human"
model = "accessories_all_together_human_tiny_model"

In [ ]:
if "gdrive" not in folder:
    folder = "/gdrive/MyDrive/"+folder

In [ ]:
def get_files(folder_path):
    files = listdir(folder_path)
    valid_path = None
    name_path = None
    weight_path = None
    for i in files:
        if "valid.txt" in i:
            valid_path = join(folder_path, i)
        elif "obj.name" in i:
            name_path = join(folder_path, i)
        elif "best.weights" in i:
            weight_path = join(folder_path, i)
    return valid_path, name_path, weight_path

In [ ]:
valid_path, name_path, weight_path = get_files(model)
valid_, name_, _ = get_files(folder)

if valid_path == None:
    if valid_ ==None:
        print("Valid file path not found")
    else:
        valid_path = valid_
        print("Valid file path selected")
else:
    print("Valid file path selected")

if name_path == None:
    if name_ ==None:
        print("obj.name file path not found")
    else:
        name_path = name_
        shutil.copy(name_, join(model, "obj.names"))
        print("obj.name file path selected")
else:
    print("obj.name file path selected")

if weight_path==None:
    print("Best weight file cannot be found.")
else:
    print("Best weight file selected.")

Valid file path selected
obj.name file path selected
Best weight file selected.


In [ ]:
#run this for getting paths of images used for training
f=open(valid_path,"r")
paths=f.read()
f.close()
paths=paths.split('\n')
shuffle(paths)
print("Total number of images found:",len(paths))
image_paths = []
for i in paths:
    if "gdrive" not in i:
        nm = "/gdrive/MyDrive/"+i[3:]
        image_paths.append(nm)
    else:
        image_paths.append(i)

Total number of images found: 472


In [ ]:
# import glob
# f1=[]
# for filename in glob.iglob(folder + '/' + '**/*.jpg', recursive=True):
#     f1.append(filename)
# len(f1)

In [ ]:
a=['scarfs', 'wallet', 'jhumkas', 'nose_ring', 'fanny', 'mang_tikka', 'nose_pin', 'wide', 'tobacco_purses', 'chandelier-earings', 'shoulder_bag', 'sunglasses',
   'waist chain', 'statement_ring', 'snood', 'hand_bag', 'bangle', 'tote', 'travel', 'hat', 'beanie', 'beret', 'brooch', 'rings', 'chokar', 'kalangi', 'turban',
   'briefcase', 'passa', 'pocket_square', 'zipper', 'chains', 'bracelet', 'sports', 'folder', 'cap', 'makeup_bag', 'dupatta', 'anklet', 'necklace', 'eyeglasses',
   'drawstring', 'watch', 'kada', 'earring', 'phone_case', 'clutch', 'slim']
print(len(a))
ind=0
for i in a:
  c=0
  for j in image_paths:
    if i in j:
      c+=1
  print('{}: {}: {}'.format(ind,i,c))
  ind=ind+1
i_p=[]
for j in image_paths:
  if a[8] in j:
    i_p.append(j)
image_paths=i_p
print(len(image_paths))

48
0: scarfs: 12
1: wallet: 13
2: jhumkas: 7
3: nose_ring: 9
4: fanny: 7
5: mang_tikka: 11
6: nose_pin: 9
7: wide: 11
8: tobacco_purses: 9
9: chandelier-earings: 15
10: shoulder_bag: 6
11: sunglasses: 7
12: waist chain: 9
13: statement_ring: 8
14: snood: 12
15: hand_bag: 17
16: bangle: 10
17: tote: 11
18: travel: 6
19: hat: 14
20: beanie: 9
21: beret: 7
22: brooch: 9
23: rings: 25
24: chokar: 9
25: kalangi: 14
26: turban: 17
27: briefcase: 6
28: passa: 3
29: pocket_square: 13
30: zipper: 14
31: chains: 11
32: bracelet: 13
33: sports: 13
34: folder: 13
35: cap: 11
36: makeup_bag: 10
37: dupatta: 6
38: anklet: 7
39: necklace: 9
40: eyeglasses: 11
41: drawstring: 8
42: watch: 5
43: kada: 8
44: earring: 9
45: phone_case: 11
46: clutch: 13
47: slim: 5
9


In [ ]:
yolo = YOLOv4(tiny=("tiny" in model))
yolo.classes = name_path
yolo.make_model()
yolo.load_weights(weight_path, weights_type="yolo")

In [ ]:
def max_depth(folder:str):
    tmp = sorted(listdir(folder))
    mx = 0
    for i in tmp:
        if "." not in i:
            mx = max(mx, 1+max_depth(join(folder, i)))
    return mx
# max_depth(folder)

In [ ]:
classDict={}
count=0
for imgpath in image_paths:
    count+=1
    try:
        original_image = cv2.imread(imgpath)
        resized_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
    except:
        print("Error occured for", imgpath)
        input("press any key")
        clear_output()
        continue
    resized_image = yolo.resize_image(resized_image)
    resized_image = resized_image / 255
    input_data = resized_image[np.newaxis].astype(np.float32)
    candidates = yolo.model.predict(input_data)

    _candidates = []
    for candidate in candidates:
        batch_size = candidate.shape[0]
        grid_size = candidate.shape[1]
        _candidates.append(          
            tf.reshape(
                candidate, shape=(1, grid_size * grid_size * 3, -1)
            )
        )
    candidates = np.concatenate(_candidates, axis=1)

    pred_bboxes = yolo.candidates_to_pred_bboxes(candidates[0],iou_threshold=0.5,score_threshold=0.2)
    # print(len(pred_bboxes))
    pred_bboxes = yolo.fit_pred_bboxes_to_original(
        pred_bboxes, original_image.shape
    )
    img=Image.open(imgpath)
    width, height = img.size
    new_height=800
    new_width=int(width*(new_height/height))
    img=img.resize((new_width,new_height))

    fig, ax = plt.subplots(figsize=(8, 8))
    ax.imshow(img, interpolation='nearest')
    plt.tight_layout()
    plt.show()
    print(count)
    predicted=list([yolo.classes[i[-2]] for i in pred_bboxes[:5]])
    print("Prediction:",predicted)
    pathToImage=imgpath.split('/')[3:-1]
    print(" -> ".join(pathToImage))
    print("-\tPress 'd' if data mismatch is there.\n-\tPress 'n' if prediction is wrong.\n-\tPress any key if prediction is right.")
    sleep(3)
    response=input()
    if "men" in pathToImage[-1].lower():
        imgBelong=pathToImage[-2]
        imgApparel=pathToImage[-1]
    else:
        imgBelong=pathToImage[-1]
        imgApparel=pathToImage[-2]

    if imgApparel not in classDict.keys():
        classDict[imgApparel]={}
    if imgBelong not in classDict[imgApparel].keys():
        classDict[imgApparel][imgBelong]={"correct":0,"total":0,"mismatch":0}
    if response=='d':
        classDict[imgApparel][imgBelong]["mismatch"]+=1
        classDict[imgApparel][imgBelong]["total"]+=1
        classDict[imgApparel][imgBelong]["correct"]+=1
    elif response=='n':
        classDict[imgApparel][imgBelong]["total"]+=1
    else:
        classDict[imgApparel][imgBelong]["total"]+=1
        classDict[imgApparel][imgBelong]["correct"]+=1
    clear_output()

In [ ]:
for i in classDict.keys():
    print("\n",i)
    for j in classDict[i].keys():
        if classDict[i][j]["total"]==0:
            print("\tNo images were there for",i)
        else:
            print(" "*4+j.ljust(10," "),"-->",classDict[i][j]["correct"]/classDict[i][j]["total"]*100,"%")


 accessories_all_together_human
    tobacco_purses --> 100.0 %


In [ ]:
print("Data Mismatch Count\n")
for i in classDict.keys():
    print("\n",i)
    for j in classDict[i].keys():
        print(" "*4,j.ljust(20," "),"-->",classDict[i][j]["mismatch"],"/",classDict[i][j]["total"])

Data Mismatch Count


 accessories_all_together_human
     tobacco_purses       --> 0 / 9
